In [ ]:
import numpy as np
from scipy.stats import t, norm

actuals = np.array([8314.98, 8987.24, 8632.06, 10867.59, 8950.49])
preds_rf = np.array([8384.87, 9189.2, 8888.62, 11120.95, 8793.22])
preds_ar = np.array([8435.78, 9236.39, 8998.56, 11298.76, 9270.62])

In [11]:

def dm_test_hln(actual, pred1, pred2, h=1, crit="MSE"):
    """
    Harvey-Leybourne-Newbold调整的Diebold-Mariano检验
    针对小样本进行了调整
    
    参数:
    actual: 真实值数组
    pred1: 模型1的预测值数组
    pred2: 模型2的预测值数组
    h: 预测范围
    crit: 损失函数标准 ("MSE", "MAPE")
    """
    # 确保输入长度一致
    if len(actual) != len(pred1) or len(actual) != len(pred2):
        raise ValueError("所有输入数组必须具有相同长度")
    
    # 计算损失函数
    if crit == "MSE":
        loss1 = (actual - pred1)**2
        loss2 = (actual - pred2)**2
    elif crit == "MAPE":
        # 避免除以零的情况
        loss1 = np.abs((actual - pred1) / np.where(actual != 0, actual, 1e-10))
        loss2 = np.abs((actual - pred2) / np.where(actual != 0, actual, 1e-10))
    else:
        raise ValueError("crit参数必须是'MSE'或'MAPE'")
    
    # 计算损失差异
    d = loss1 - loss2
    n = len(d)
    
    # 计算差异的均值
    mean_d = np.mean(d)
    
    # 计算差异的方差（考虑自相关性）
    autocov = []
    for lag in range(0, h):
        if lag == 0:
            autocov.append(np.var(d, ddof=0))  # 使用有偏估计
        else:
            if n - lag > 0:  # 确保有足够的数据点计算协方差
                autocov.append(np.cov(d[lag:], d[:n-lag], bias=True)[0, 1])
            else:
                autocov.append(0)  # 如果没有足够数据点，设协方差为0
    
    # 计算方差估计
    var_d = (autocov[0] + 2 * sum(autocov[1:])) / n if n > 0 else 0
    
    # 计算原始DM统计量
    dm_stat = mean_d / np.sqrt(var_d) if var_d > 0 else 0
    
    # HLN调整因子
    hln_adj = np.sqrt((n + 1 - 2*h + h*(h-1)/n) / n) if n > 0 else 1
    
    # 应用HLN调整
    dm_stat_adj = dm_stat * hln_adj
    
    # 计算p值（使用t分布，自由度为n-1）
    p_value = 2 * t.cdf(-abs(dm_stat_adj), df=n-1) if n > 1 else 1.0
    
    return dm_stat_adj, p_value, mean_d

# 您的数据
actuals = np.array([8314.98, 8987.24, 8632.06, 10867.59, 8950.49])
preds_rf = np.array([8384.87, 9189.2, 8888.62, 11120.95, 8793.22])
preds_ar = np.array([8435.78, 9236.39, 8998.56, 11298.76, 9270.62])

print("DM Test Results Comparison (h=1, HLN Adjusted)")
print("=" * 50)

# Calculate and display DM test results based on MSE
dm_stat_mse, p_value_mse, mean_diff_mse = dm_test_hln(actuals, preds_rf, preds_ar, h=1, crit="MSE")
print("DM Test based on MSE:")
print(f"  Adjusted DM Statistic: {dm_stat_mse:.4f}")
print(f"  P-value: {p_value_mse:.4f}")
print(f"  Mean Loss Difference (RF - AR): {mean_diff_mse:.2f}")

# Calculate and display DM test results based on MAPE
dm_stat_mape, p_value_mape, mean_diff_mape = dm_test_hln(actuals, preds_rf, preds_ar, h=1, crit="MAPE")
print("\nDM Test based on MAPE:")
print(f"  Adjusted DM Statistic: {dm_stat_mape:.4f}")
print(f"  P-value: {p_value_mape:.4f}")
print(f"  Mean Loss Difference (RF - AR): {mean_diff_mape:.4f}")

# Calculate and display basic error metrics
mse_rf = np.mean((actuals - preds_rf)**2)
mse_ar = np.mean((actuals - preds_ar)**2)
mape_rf = np.mean(np.abs((actuals - preds_rf) / np.where(actuals != 0, actuals, 1e-10))) * 100
mape_ar = np.mean(np.abs((actuals - preds_ar) / np.where(actuals != 0, actuals, 1e-10))) * 100

print("\nBasic Error Metrics:")
print(f"  Random Forest MSE: {mse_rf:.2f}")
print(f"  AR(3) Model MSE: {mse_ar:.2f}")
print(f"  Random Forest MAPE: {mape_rf:.2f}%")
print(f"  AR(3) Model MAPE: {mape_ar:.2f}%")

# Result interpretation
alpha = 0.05
print("\nResult Interpretation:")
print("=" * 50)

def interpret_result(dm_stat, p_value, criterion):
    if p_value < alpha:
        if dm_stat < 0:
            return f"Based on {criterion}, Random Forest is significantly better than AR(3) (p < {alpha})"
        else:
            return f"Based on {criterion}, AR(3) is significantly better than Random Forest (p < {alpha})"
    else:
        return f"Based on {criterion}, no significant difference between models (p ≥ {alpha})"

print(interpret_result(dm_stat_mse, p_value_mse, "MSE"))
print(interpret_result(dm_stat_mape, p_value_mape, "MAPE"))


DM Test Results Comparison (h=1, HLN Adjusted)
DM Test based on MSE:
  Adjusted DM Statistic: -2.9486
  P-value: 0.0420
  Mean Loss Difference (RF - AR): -59792.15

DM Test based on MAPE:
  Adjusted DM Statistic: -4.4718
  P-value: 0.0111
  Mean Loss Difference (RF - AR): -0.0117

Basic Error Metrics:
  Random Forest MSE: 40084.13
  AR(3) Model MSE: 99876.28
  Random Forest MAPE: 2.03%
  AR(3) Model MAPE: 3.20%

Result Interpretation:
Based on MSE, Random Forest is significantly better than AR(3) (p < 0.05)
Based on MAPE, Random Forest is significantly better than AR(3) (p < 0.05)
